P317 Example 8.1

In [1]:
import numpy as np

In [2]:
N = 3
K = 3
mu = np.array([0.8, 0.6, 0.4])
m = np.array([2, 3, 1])
alpha = np.array([1.0, 0.667, 0.2])
rho = alpha / mu

In [3]:
def fact(n):
    if n <= 1:
        return 1
    return n * fact(n-1)

In [4]:
def getBeta(n, k): #P289 式 7.62
    if m[n] == 1:
        return 1
    elif k <= m[n]:
        return fact(k)
    elif k >= m[n]:
        return fact(m[n]) * m[n]**(k-m[n])

In [5]:
def getF(n, k):#n:ノード番号は1引いた数字
    f = (alpha[n]/mu[n])**k
    f /= getBeta(n, k)
    return f

In [6]:
for n in range(N):
    for k in range(K+1): 
        print('n = {0}, k = {1}, F = {2}'.format(n, k, getF(n, k)))

n = 0, k = 0, F = 1.0
n = 0, k = 1, F = 1.25
n = 0, k = 2, F = 0.78125
n = 0, k = 3, F = 0.48828125
n = 1, k = 0, F = 1.0
n = 1, k = 1, F = 1.1116666666666668
n = 1, k = 2, F = 0.617901388888889
n = 1, k = 3, F = 0.22896679243827167
n = 2, k = 0, F = 1.0
n = 2, k = 1, F = 0.5
n = 2, k = 2, F = 0.25
n = 2, k = 3, F = 0.125


In [7]:
def calcConvolution(n, k, rho): #メモ化利用なし
    #rho = alpha / mu
    if k == 0: #Gj(0) = 1 (j = 1,...,N)
        g = 1
    elif n == 0:#G(0,k) = f0(k)
        g = np.power(rho[n], k)
    else : 
        g = calcConvolution(n-1, k,rho) + rho[n] * calcConvolution(n, k-1,rho)
    return g

In [8]:
g = calcConvolution(N-1, K, rho) #ノード番号は1小さくしている
print(g)
#窓口数を考慮できないので値が合わない

9.418018115740743


In [9]:
def getConvolution(n, k, rho): #QN&MC P314 式(8.4),(8.5),(8.6)利用
    if k == 0:
        g = 1
    elif n == 0:
        g = getF(n, k)
    else :
        g = 0
        for i in range(k + 1):
            g += getF(n, i) * getConvolution(n-1, k-i, rho)
    #print('n = {0}, k = {1}, G(n,k) = {2}'.format(n, k, g)) #QN&MC P314 Table8.1
    return g

In [10]:
g = getConvolution(N-1, K, rho) #ノード番号は1小さくしている
print(g)

4.467898389660494


In [11]:
def getMarginalProbabilities(n, k): #QN&MC P316 式(8.13)
    pi = rho[n]**k / getConvolution(N-1, K, rho) * (getConvolution(N-1, K-k, rho) - rho[n] * getConvolution(N-1, K-k-1, rho))
    return pi

In [12]:
for k in range(K+1): #n = Nのとき
    marginal = getMarginalProbabilities(N-1, k) #ノード番号は1小さくしている
    print('pi({0},{1}) = {2}'.format(n, k, marginal))

pi(2,0) = 0.5277905082487573
pi(2,1) = 0.31208573685066865
pi(2,2) = 0.13214639527904107
pi(2,3) = 0.02797735962153304


In [13]:
def getGi(n, k, rho): #QN&MC P316 式(8.11), (8.12)
    if k == 0: #式(8.12)
        gi = 1
    else:
        gi = getConvolution(N-1, k, rho)
        for i in range(1, k + 1):
            gi -= getF(n, i) * getGi(n, k-i, rho)
    return gi

In [14]:
for n in range(N-1): #QN&MC P319確認
    for k in range(K+1):
        gi = getGi(n, k, rho)
        print('G({0},{1}) = {2}'.format(n, k, gi))

G(0,0) = 1
G(0,1) = 1.6116666666666668
G(0,2) = 1.423734722222222
G(0,3) = 0.9408341535493832
G(1,0) = 1
G(1,1) = 1.75
G(1,2) = 1.6562499999999996
G(1,3) = 1.3164062500000002


In [15]:
def getMarginal(): #QN&MC P315 式(8.7)
    mpi = np.zeros((N, K+1)) #周辺分布
    l = np.zeros(N) #平均系内人数
    for n in range(N):#周辺分布計算
        for k in range(K+1):
            mpi[n,k] = getF(n, k) * getGi(n, K - k, rho) / g
    for n in range(N):
        for k in range(K+1):
            l[n] += k * mpi[n,k]
    return mpi, l

In [16]:
mpi, l = getMarginal()
print(mpi)
print(l)

[[0.21057644 0.39832338 0.28181361 0.10928656]
 [0.29463657 0.41209485 0.24202149 0.05124709]
 [0.52779051 0.31208574 0.1321464  0.02797736]]
[1.28981029 1.0498791  0.66031061]


P323 Example 8.2

In [17]:
N = 4
R = 2
K1 = 1
K2 = 2
mu = np.array([[1/1, 1/2],[1/4, 1/5],[1/8, 1/10],[1/12, 1/16]])
alpha = np.array([[1, 1],[0.4, 0.4],[0.4, 0.3],[0.2, 0.3]])
#Node1:Type2プロセッサシェアリング（Processor Sharing: PS）, Node2:Type4後着順割込継続型（LCFS-PR), Node3:Type4後着順割込継続型（LCFS-PR), Node4:Type3無限サーバ（Infinite Server: IS）, その他Type1(FCFS) 

In [45]:
# P323 Table 8.2 Fi(Si)の計算, P303式(7.82)
#Siはノードiにおけるクラス別人数分布:(ノードiのクラス0の人数, ノードiのクラス1の人数)
#Siは(0,0),(1,0),(0,1),(1,1),(0,2),(1,2) K1=1,K2=2なので
def FS(n, alpha, mu, Si, type_number):#sは状態分布, type_number =1(FCFS),2(PS),3(IS),4(LCFS-PR)
    f = 1
    if type_number == 1:
        print('FCFS')
    else:
        for r in range(R):#type-3はこのループで終わり
            f *= 1 / fact(Si[r]) * (alpha[n,r] / mu[n,r])**Si[r]
        if type_number == 2 or type_number == 4:#type-2,4は累乗をかける    
            f *= fact(sum(Si))
    return f

In [51]:
type_number = [2, 4, 4, 3]
for n in range(N):
    for k1 in range(K1+1): #kのリストを作成
        for k2 in range(K2+1):
            f = FS(n, alpha, mu, [k1, k2], type_number[n])
            print('F{0}([{1},{2}]) = {3}'.format(n, k1, k2, f))

F0([0,0]) = 1.0
F0([0,1]) = 2.0
F0([0,2]) = 4.0
F0([1,0]) = 1.0
F0([1,1]) = 4.0
F0([1,2]) = 12.0
F1([0,0]) = 1.0
F1([0,1]) = 2.0
F1([0,2]) = 4.0
F1([1,0]) = 1.6
F1([1,1]) = 6.4
F1([1,2]) = 19.200000000000003
F2([0,0]) = 1.0
F2([0,1]) = 2.9999999999999996
F2([0,2]) = 8.999999999999998
F2([1,0]) = 3.2
F2([1,1]) = 19.2
F2([1,2]) = 86.39999999999999
F3([0,0]) = 1.0
F3([0,1]) = 4.8
F3([0,2]) = 11.52
F3([1,0]) = 2.4000000000000004
F3([1,1]) = 11.520000000000001
F3([1,2]) = 27.648000000000003
